<a href="https://colab.research.google.com/github/RafaelCaballero/Julio24/blob/main/code/19cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la ciencia de datos con Python
### Rafa Caballero


## Validación cruzada

Como hemos visto para obtener resultados fiables tenemos que repetir muchas veces un experimento. Una forma de evitar esto es utilizar **validación cruzada**. La idea es la siguiente

- El dataset se divide aleatoriamente en n trozos (n es un número decidido por el usuario)
- Se realizan n experimentos. En cada experimento uno de los trozos es seleccionado como test, y el resto como entrenamiento
- El resultado es la media de las n iteraciones

El encargado de esta labor es el método [sklearn.model_selection.cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score) que recibe el método, la métrica y el número de divisiones a realizar. Las métricas posibles se pueden encontrar [aquí](https://scikit-learn.org/stable/modules/model_evaluation.html)

In [ ]:
# Carga del fichero
import pandas as pd
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/pisaDataClean.csv"
df = pd.read_csv(url)
df

In [ ]:
import math
from  sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression


# 1 selección de datos
XColumns = ["SCI", "REA"]
yColumn = "MAT"
X = df[XColumns]
y = df[yColumn]

# 2,3,4
metodo = LinearRegression()
res = cross_val_score(metodo, X, y, cv=10, scoring="neg_mean_squared_error")
math.sqrt(-1*res.mean())

In [ ]:
res

Puede ser muy lento pero puede trabajar en paralelo; `n_jobs=1` indica que utilice todos los procesadores disponibles

In [ ]:
metodo = LinearRegression()
res = cross_val_score(metodo, X, y, cv=10, scoring="neg_mean_squared_error",verbose=3,n_jobs=-1)

En algunos casos no tendremos un método "scoring" adecuado, o queerremos hacer particiones más complejas, en ese caso se puede construir la métrica o el método para dividir. Por ejemplo, podemos querer "barajar" antes de dividir en trozos:

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

scorer = make_scorer(mean_squared_error)
particiones = KFold(n_splits=10, shuffle=True) # nuevo. Para clas. KFoldStratified
res = cross_val_score(metodo,X,y,scoring=scorer,cv=particiones)
math.sqrt(res.mean())


Si queremos además hacerla media de varias pruebas podemos usar:

In [ ]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

repite = RepeatedKFold(n_splits=20, n_repeats=10)
res = cross_val_score(metodo,X,y,scoring=scorer,cv=repite)
RMSE = math.sqrt(res.mean())
RMSE

Si lo que queremos es obtener una buena estimación de las predicciones totales podemos utilizar [cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)

In [ ]:
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(metodo, X, y, cv=len(X))

In [ ]:
import matplotlib.pyplot as plt


x = range(len(y))
fig, ax = plt.subplots(figsize=(10, 5))
ci = 1.96*RMSE
for i in range(len(y_pred)):
    plt.plot([x[i],x[i]], [y_pred[i],y[i]],color="blue",alpha=.4)
ax.fill_between(x, ( y_pred-ci), ( y_pred+ci), color='b', alpha=.1)
ax.scatter(x,y_pred,color="red",s=8,label="predicho")
ax.scatter(x,y,color="green",s=8,label="real")
plt.legend()
plt.show()

Mas información: https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation